In [23]:
import torch
import os

**Prepare dataset**

In [18]:
%%capture
% cd ch2
!python download.py

# Takes ~ 1 hour

In [21]:
% cd ../

/home/grigoriy/Repos/Book_Ian_Pointer_PyTorch_for_DL


Remove 0 bytes images

In [32]:
files = [os.path.join(r,file) for r,d,f in os.walk("./ch2") for file in f]
files[:5]

['./ch2/images.csv',
 './ch2/download.py',
 './ch2/train/cat/1010884970_1aa4af4701.jpg',
 './ch2/train/cat/10298902.jpg',
 './ch2/train/cat/54461669_2791e8f95e.jpg']

In [34]:
for file in files:
    if os.path.getsize(file) == 0:
        os.remove(file)

In [35]:
TRAIN_PATH = './ch2_data/train/'
VAL_PATH = './ch2_data/val/'
TEST_PATH = './ch2_data/test/'